# ReACT - Synergizing reasoning and acting in language models
1. LLM thinks about what to do and then decide the action to take
2. action is taken in environment and observation is returned
3. with that observation the LLM repeats

https://til.simonwillison.net/llms/python-react-pattern

## OpenAI usage

In [4]:
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:1234/v1", 
    api_key="dummy"
)

response = client.chat.completions.create(
    model="qwen/qwen3-4b-2507",
    messages=[{"role": "user", "content": "What is the capital of France?"}]
)

print(response.choices[0].message.content)

The capital of France is Paris.


## ReACT Python Agent

In [11]:
def average_dog_weight(breed: str) -> str:
    if breed == "Bulldog":
        return "Bulldog weighs on average51 lbs"
    elif breed == "Collie":
        return "Collie weighs on average 55 lbs"
    else:
        return "An average dog weighs on average 51 lbs"


def calculate(expression: str) -> str:
    return eval(expression)

known_actions = {
    "average_dog_weight": average_dog_weight,
    "calculate": calculate
}

In [6]:
class Agent:
    def __init__(self, system_prompt: str):
        self.system_prompt = system_prompt
        self.messages = []
        self.messages.append({"role": "system", "content": system_prompt})

    def __call__(self, input: str) -> str:
        self.messages.append({"role": "user", "content": input})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self) -> str:
        response = client.chat.completions.create(
            model="qwen/qwen3-4b-2507",
            messages=self.messages
        )
        return response.choices[0].message.content


react_system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
Returns average dog weight of the given breed


Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight action
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A bulldog weighs 51 lbs

You then output:

Answer: A bulldog weighs 51 lbs
""".strip()

agent = Agent(system_prompt=react_system_prompt)

In [7]:
agent("How much does a Bulldog weigh?")

"Thought: I should look up the dog's weight using the average_dog_weight action.  \nAction: average_dog_weight: Bulldog  \nPAUSE"

In [10]:
action_result = average_dog_weight("Bulldog")
agent(f"Observation: ${action_result}")


'Answer: A bulldog weighs 51 lbs'

In [ ]:
import re
from typing import Tuple

action_re = re.compile(r"^Action: (\w+): (.*)$")

def contains_action(result: str) -> bool:
    return "Action:" in result

def extract_action(result: str) -> Tuple[str, str]:
    actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
    if actions:
        return actions[0].groups()
    return None

def agent_loop(query: str, max_turns: int = 10) -> str:
    agent = Agent(system_prompt=react_system_prompt)
    prompt = query
    for _ in range(max_turns):
        result = agent(prompt)
        print('log:', result)
        if contains_action(result):
            action, action_intput = extract_action(result)
            print(f"log: -- running {action} with {action_intput}")
            action_result = known_actions[action](action_intput)
            prompt = f"Observation: {action_result}"
        else:
            return result
    return None

agent_loop("How much does a Bulldog weigh?")

log: Thought: I should look up the dog's weight using the average_dog_weight action.  
Action: average_dog_weight: Bulldog  
PAUSE
log: -- running average_dog_weight with Bulldog  
log: Answer: A bulldog weighs 51 lbs


'Answer: A bulldog weighs 51 lbs'

In [18]:
agent_loop("How much does a Bulldog and Collie weigh together?")

log: Thought: I need to find the average weights of both a Bulldog and a Collie, then add them together. I'll start by getting the average weight of a Bulldog using the average_dog_weight action.
Action: average_dog_weight: Bulldog
PAUSE
log: -- running average_dog_weight with Bulldog
log: Thought: Now that I have the average weight of a Bulldog, I need to get the average weight of a Collie next. I'll use the average_dog_weight action for Collie.  
Action: average_dog_weight: Collie
PAUSE
log: -- running average_dog_weight with Collie
log: Thought: Now that I have both average weights — Bulldog at 51 lbs and Collie at 55 lbs — I can calculate their total weight by adding them together.  
Action: calculate: 51 + 55
PAUSE
log: -- running calculate with 51 + 55
log: Answer: A Bulldog and a Collie together weigh 106 lbs.


'Answer: A Bulldog and a Collie together weigh 106 lbs.'